In [1]:
import pandas as pd
from geopandas import GeoDataFrame, read_file

import sys
sys.path.append("..")
import movingpandas as mpd
mpd.show_versions()

import warnings
warnings.simplefilter("ignore")


MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.9.19 | packaged by conda-forge | (main, Mar 20 2024, 12:50:21)  [GCC 12.3.0]
executable : /home/arg/miniconda3/envs/movingpandas/bin/python
machine    : Linux-5.15.0-113-generic-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.8.4
GDAL data dir: /home/arg/miniconda3/envs/movingpandas/share/gdal
PROJ       : 9.3.1
PROJ data dir: /home/arg/miniconda3/envs/movingpandas/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.3
pandas     : 2.2.1
fiona      : 1.9.6
numpy      : 1.26.4
shapely    : 2.0.3
rtree      : 1.2.0
pyproj     : 3.6.1
matplotlib : 3.8.3
mapclassify: 2.6.1
geopy      : 2.4.1
holoviews  : 1.18.3
hvplot     : 0.9.2
geoviews   : 1.11.1
stonesoup  : 1.2


/home/arg/miniconda3/envs/movingpandas/lib/python3.9/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
import csv
import os
import shutil
import xml.etree.ElementTree as ET
from datetime import timedelta

import numpy as np
from dateutil import parser


def generate_grid_and_path(x_min, x_max, x_n, y_min, y_max, y_n):
    """
    Generates a grid and a path on the grid.

    Parameters:
    - x_min (float): The minimum x value of the grid.
    - x_max (float): The maximum x value of the grid.
    - x_n (int): The number of points in the x direction.
    - y_min (float): The minimum y value of the grid.
    - y_max (float): The maximum y value of the grid.
    - y_n (int): The number of points in the y direction.

    Returns:
    - path (list): A list of coordinates representing the path on the grid.
    """

    x_values = np.linspace(x_min, x_max, x_n)
    y_values = np.linspace(y_min, y_max, y_n)
    X, Y = np.meshgrid(x_values, y_values)
    path = []

    for i in range(x_n):
        for j in range(y_n):
            if i % 2 == 0:
                path.append((X[j, i], Y[j, i]))
            else:
                path.append((X[-j - 1, i], Y[-j - 1, i]))

    if x_n % 2 == 0:
        for j in range(y_n):
            for i in range(x_n):
                if j % 2 == 0:
                    path.append((X[j, i], Y[j, i]))
                else:
                    path.append((X[j, -i - 1], Y[j, -i - 1]))
    else:
        for j in range(y_n - 1, -1, -1):
            for i in range(x_n):
                if j % 2 == 0:
                    path.append((X[j, i], Y[j, i]))
                else:
                    path.append((X[j, -i - 1], Y[j, -i - 1]))
    return path


# def parse_corrdinates_str_list(kml_path):
#     """
#     Parses a KML file and extracts the coordinates from the LineString elements.

#     Args:
#         kml_path (str): The path to the KML file.

#     Returns:
#         list: A list of coordinate strings extracted from the LineString elements.
#     """
#     namespaces = {
#         'kml': 'http://www.opengis.net/kml/2.2',
#         'gx': 'http://www.google.com/kml/ext/2.2',
#         'atom': 'http://www.w3.org/2005/Atom'
#     }
#     tree = ET.parse(kml_path)
#     root = tree.getroot()
#     coordinates = None
#     coordinates_str_list = []
#     for placemark in root.findall(".//Placemark"):
#         name = placemark.find("name")
#         if name is not None :
#             # Find the LineString element
#             line_string = placemark.find(".//LineString")
#             if line_string is not None:
#                 # Extract the coordinates
#                 coordinates = line_string.find("coordinates")
#                 if coordinates is not None:
#                     print("find!!!")
#                     coordinates_str_list.append(coordinates.text.strip())
#     print("coordinates_str_list: ", coordinates_str_list)
#     return coordinates_str_list

def parse_coordinates_str_list(kml_path):
    """
    Parses a KML file and extracts the coordinates from the LineString elements.

    Args:
        kml_path (str): The path to the KML file.

    Returns:
        list: A list of coordinate strings extracted from the LineString elements.
    """
    namespaces = {
        'kml': 'http://www.opengis.net/kml/2.2',
        'gx': 'http://www.google.com/kml/ext/2.2',
        'atom': 'http://www.w3.org/2005/Atom'
    }
    
    tree = ET.parse(kml_path)
    root = tree.getroot()
    coordinates_str_list = []

    for placemark in root.findall(".//kml:Placemark", namespaces):
        name = placemark.find("kml:name", namespaces)
        if name is not None and name.text == "1 Flight Path  Stabilize":
            line_string = placemark.find(".//kml:LineString", namespaces)
            if line_string is not None:
                coordinates = line_string.find("kml:coordinates", namespaces)
                if coordinates is not None:
                    coordinates_str_list.append(coordinates.text.strip())
                    # print("find!!!")
                    
    # print("coordinates_str_list: ", coordinates_str_list)
    return coordinates_str_list

def get_means(coordinates):
    """
    Calculate the mean x and y coordinates from a list of coordinates.

    Parameters:
    coordinates (list): A list of coordinate tuples, where each tuple contains x and y coordinates.

    Returns:
    tuple: A tuple containing the mean x and y coordinates.

    """
    mean_x = np.mean([coord[0] for coord in coordinates])
    mean_y = np.mean([coord[1] for coord in coordinates])
    return mean_x, mean_y


def write_to_csv(
    csv_file_path, coordinates, trajectory_id, tracker, base_timestamp, mean_x, mean_y, grid_size_half=10e-5, xy_n=10
):
    """
    Write coordinates to a CSV file.

    Args:
        csv_file_path (str): The path to the CSV file.
        coordinates (list): A list of coordinate tuples (x, y, z).
        trajectory_id (int): The ID of the trajectory.
        tracker (str): The tracker name.
        base_timestamp (str): The base timestamp for generating timestamps.
        mean_x (float): The mean x-coordinate.
        mean_y (float): The mean y-coordinate.
        grid_size_half (float, optional): Half of the grid size. Defaults to 10e-5.
        xy_n (int, optional): The number of points in the x and y directions. Defaults to 10.

    Returns:
        None
    """
    base_time = parser.parse(base_timestamp)
    with open(csv_file_path, "w", newline="") as csv_file:
        writer = csv.writer(csv_file, delimiter=";")

        headers = ["X", "Y", "fid", "id", "sequence", "trajectory_id", "tracker", "t"]
        writer.writerow(headers)
        for i, coord in enumerate(coordinates, start=1):
            x, y, _ = coord
            timestamp = (base_time + timedelta(seconds=1 * (i - 1))).isoformat()
            row = [x, y, i, i, i, trajectory_id, tracker, timestamp]
            writer.writerow(row)

        path = generate_grid_and_path(
            mean_x - grid_size_half,
            mean_x + grid_size_half,
            xy_n,
            mean_y - grid_size_half,
            mean_y + grid_size_half,
            xy_n,
        )
        for i, coord in enumerate(path, start=len(coordinates) + 1):
            x, y = coord
            timestamp = (base_time + timedelta(seconds=1 * (i - 1))).isoformat()
            row = [x, y, i, i, i, trajectory_id + 1, tracker, timestamp]
            writer.writerow(row)

    print(f"Saved to {csv_file_path}")


def convert_kml_to_csv(kml_path):
    """
    Converts a KML file to a CSV file.

    Args:
        kml_path (str): The path to the KML file.

    Returns:
        None
    """
    kml_name = kml_path.split("/")[-1].split(".")[0]

    #print(parse_corrdinates_str_list(kml_path))
    coordinates_str_list = parse_coordinates_str_list(kml_path)[0].split()
    coordinates = [[float(coord) for coord in coord_str.split(",")] for coord_str in coordinates_str_list]
    mean_x, mean_y = get_means(coordinates)
    print(f"Min x: {min([coord[0] for coord in coordinates])}, Min y: {min([coord[1] for coord in coordinates])}")
    print(f"Max x: {max([coord[0] for coord in coordinates])}, Max y: {max([coord[1] for coord in coordinates])}")
    xy_n = 61
    grid_size_half = 1 / 111_111 * xy_n * 4 # Convert to meter

    print(f"Mean x: {mean_x}, Mean y: {mean_y}")

    trajectory_id = 1
    tracker = 19
    base_timestamp = "2008-12-11 04:42:14+00"

    csv_file_path = f"./data/{kml_name}.csv"
    
    write_to_csv(
        csv_file_path,
        coordinates,
        trajectory_id,
        tracker,
        base_timestamp,
        mean_x,
        mean_y,
        grid_size_half,
        xy_n=xy_n,
    )
    # shutil.copy(csv_file_path, os.path.expanduser("./data/"))
    print(f"Copied to ./data/{kml_name}.csv")


# kml_pathes = [

#     "飛行軌跡_20230716070634_R8039938565.kml",
#     "飛行軌跡_20230716070703_R8059961112.kml",
#     "飛行軌跡_20230716071226_R6234489650.kml",
#     "飛行軌跡_20230716071856_R6311160153.kml",
#     "飛行軌跡_20230716073025_R6965064834.kml"
# ]

kml_pathes = [

    "ardupilot-logs/kmls/log_4_2024-7-6-16-49-04.kml",
    "ardupilot-logs/kmls/log_5_2024-7-7-17-00-02.kml",
    "ardupilot-logs/kmls/log_6_2024-7-8-16-09-54.kml",
    "ardupilot-logs/kmls/log_7_2024-7-10-16-07-10.kml",
]


kml_pathes = [f"./data/{kml_path}" for kml_path in kml_pathes]

for kml_path in kml_pathes:
    convert_kml_to_csv(kml_path)

Min x: 120.1541691, Min y: 22.9891336
Max x: 120.1566023, Max y: 22.9918357
Mean x: 120.15562838756955, Mean y: 22.989727834242785
Saved to ./data/log_4_2024-7-6-16-49-04.csv
Copied to ./data/log_4_2024-7-6-16-49-04.csv
Min x: 120.1542372, Min y: 22.9891215
Max x: 120.1568408, Max y: 22.9899342
Mean x: 120.1557371241713, Mean y: 22.989426902508843
Saved to ./data/log_5_2024-7-7-17-00-02.csv
Copied to ./data/log_5_2024-7-7-17-00-02.csv
Min x: 120.154224, Min y: 22.9892098
Max x: 120.1564775, Max y: 22.9903752
Mean x: 120.15533520336521, Mean y: 22.98950685584751
Saved to ./data/log_6_2024-7-8-16-09-54.csv
Copied to ./data/log_6_2024-7-8-16-09-54.csv
Min x: 120.1545826, Min y: 22.9887741
Max x: 120.1565143, Max y: 22.9901351
Mean x: 120.15553936455454, Mean y: 22.98943727115445
Saved to ./data/log_7_2024-7-10-16-07-10.csv
Copied to ./data/log_7_2024-7-10-16-07-10.csv


In [3]:
import geoviews as gv

bing_maps_tile_source = gv.tile_sources.WMTS("http://ecn.t3.tiles.virtualearth.net/tiles/a{q}.jpeg?g=1")
hvplot_defaults = {'tiles': bing_maps_tile_source, 'frame_height': 700, 'frame_width': 700, 'colorbar': True}

def plot_trajectory_on_map():
    """
    Plot trajectory on a map using GeoViews.

    This function sets up the necessary tile source from Bing Maps and defines default plot settings.

    Returns:
        None
    """
    # Your code here

plot_trajectory_on_map()


In [4]:
%%time 
df = pd.read_csv('data/log_4_2024-7-6-16-49-04.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 1.01 s, sys: 9.01 ms, total: 1.02 s
Wall time: 1.02 s


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

In [5]:
%%time 
df = pd.read_csv('data/log_5_2024-7-7-17-00-02.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 1.16 s, sys: 16.2 ms, total: 1.18 s
Wall time: 1.18 s


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

In [6]:
%%time 
df = pd.read_csv('data/log_6_2024-7-8-16-09-54.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 1.06 s, sys: 4.19 ms, total: 1.06 s
Wall time: 1.06 s


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

In [7]:
%%time 
df = pd.read_csv('data/log_7_2024-7-10-16-07-10.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 1.14 s, sys: 4.63 ms, total: 1.14 s
Wall time: 1.14 s


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)